In [1]:
from stocktrends import Renko
import yfinance as yf

tickers=['AMZN','GOOG','TSLA']
ohlcv_data={}
hour_data={}
renko_data={}
for ticker in tickers:
    temp=yf.download(ticker,period='1mo',interval='5m')
    temp.dropna(how='any',inplace=True)
    ohlcv_data[ticker]=temp
    temp=yf.download(ticker,period='1y',interval='1h')
    temp.dropna(how='any',inplace=True)
    hour_data[ticker]=temp

def ATR(DF,n=14):
    df=DF.copy()
    df['H-L']=df['High']-df['Low']
    df['H-PC']=df['High']-df['Adj Close'].shift(1)
    df['L-PC']=df['Low']-df['Adj Close'].shift(1)
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR']=df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [2]:
def renko_DF(DF, hourly_df):
    df = DF.copy()
    df.drop('Close',axis=1,inplace=True)
    df.reset_index(inplace=True)
    df.columns = ['date','open','high','low','close','volume']
    df2 = Renko(df)
    df2.brick_size = 3*round(ATR(hourly_df,120).iloc[-1],0)
    renko_df = df2.get_ohlc_data()
    return renko_df

In [3]:
for ticker in ohlcv_data:
    renko_data[ticker] = renko_DF(ohlcv_data[ticker],hour_data[ticker])

In [4]:
ohlcv_data['AMZN']

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-04-08 09:30:00-04:00,186.985992,187.289993,185.850006,185.880005,185.880005,2867480
2024-04-08 09:35:00-04:00,185.865005,186.523499,185.442001,185.499695,185.499695,1113031
2024-04-08 09:40:00-04:00,185.500000,186.149994,185.328995,186.000000,186.000000,906746
2024-04-08 09:45:00-04:00,186.020004,186.149994,185.914993,186.115005,186.115005,761720
2024-04-08 09:50:00-04:00,186.119995,186.270004,185.883804,186.046707,186.046707,896207
...,...,...,...,...,...,...
2024-05-06 15:35:00-04:00,187.710007,187.895004,187.695007,187.824997,187.824997,402155
2024-05-06 15:40:00-04:00,187.830002,187.925003,187.750000,187.860001,187.860001,377754
2024-05-06 15:45:00-04:00,187.861298,188.079895,187.729996,188.065002,188.065002,715488


In [11]:
renko_data['AMZN']

,date,open,high,low,close,uptrend
0,2023-08-07 09:30:00-04:00,135.0,138.0,135.0,138.0,True
1,2023-08-07 12:25:00-04:00,138.0,141.0,138.0,141.0,True
2,2023-08-17 09:50:00-04:00,138.0,138.0,135.0,135.0,False
3,2023-08-18 09:30:00-04:00,135.0,135.0,132.0,132.0,False
4,2023-08-31 10:00:00-04:00,135.0,138.0,135.0,138.0,True
